## Setup everything

In [2]:
import logging

# Disable all the verbose logging
logging.getLogger("litellm").setLevel(logging.WARNING)
logging.getLogger("LiteLLM").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

# Optionally, if you want to keep only critical errors:
# logging.disable(logging.CRITICAL)

In [3]:
import sys
sys.path.append('..')

In [4]:
from lib import search_results, get_article_text, setup_db

In [5]:
c = setup_db()

In [6]:
import dspy
from dotenv import load_dotenv
import tqdm

load_dotenv()

gpt41mini = dspy.LM("openai/gpt-4.1-mini")
gpt41 = dspy.LM("openai/gpt-4.1")
groq = dspy.LM("groq/llama-3.1-8b-instant")
kimi = dspy.LM("groq/moonshotai/kimi-k2-instruct")

dspy.configure(
    lm=kimi
)

In [7]:
assert kimi("hello world")

## Data Loading

In [8]:
import pandas as pd
test_set = pd.read_csv("../test_queries.tsv", sep="\t")
test_set.head()

,questions,topics,urls
0,how does a hepatic adenoma look like on MRI?,hepatic adenoma,https://radiopaedia.org/articles/hepatic-adeno...
1,penumbra ischemic stroke perfuson imaging,penumbra perfusion imaging,https://radiopaedia.org/articles/ct-perfusion-...
2,how to differentiate between a hepatic adenoma...,"hepatic adenoma, focular nodular hyperplasia",https://radiopaedia.org/articles/hepatic-adeno...
3,how does japanese encephalitis look like on MRI?,japanese encephalitis,https://radiopaedia.org/articles/japanese-ence...
4,what is CLIPPERS?,CLIPPERS,https://radiopaedia.org/articles/chronic-lymph...


In [10]:
examples = []

for row in test_set.itertuples():
    examples.append(dspy.Example(user_query=row.questions, urls=row.urls.split(","), main_topics=row.topics.split(",")).with_inputs("user_query"))

In [11]:
from sklearn.model_selection import train_test_split

train_examples, val_examples = train_test_split(examples, train_size=0.2, test_size=0.8, random_state=42)

## Model definition

In [12]:
class SearchQuerySig(dspy.Signature): 
    """Extract one or two main topics (diseases, procedures, phenomenon etc.) from the user query. 
    
    Do not include imaging modalities (CT, MRI etc.) in your topics."""

    user_query: str = dspy.InputField()
    main_topics: list[str] = dspy.OutputField()

class AnswerQuerySig(dspy.Signature):
    """Answer the user query based on the context. 
    
    If the context is not relevant to the user query, say that you don't know. Do not hallucinate answers."""

    user_query: str = dspy.InputField()
    context: list[str] = dspy.InputField()
    answer: str = dspy.OutputField(desc="concise, yet educational, markdown-styled answer to the user query.")


class RadiopaediaArticleFinder(dspy.Module):
    def __init__(self):
        self.generate_search_query = dspy.ChainOfThought(SearchQuerySig)
        
    def forward(self, user_query: str):
        topics = self.generate_search_query(user_query=user_query).main_topics
        filtered_topics = [t for t in topics if not any(modality.lower() in t.lower() 
                          for modality in ["CT", "MRI", "X-ray", "ultrasound"])]
        
        urls = []
        for topic in filtered_topics[:2]:
            results = search_results(search_term=topic, cursor=c)
            urls.extend(f"https://radiopaedia.org{r['href']}" for r in results[:2])
        
        return dspy.Prediction(urls=urls, main_topics=filtered_topics)

class RadiopaediaQA(dspy.Module):
    def __init__(self, article_finder):
        self.find_articles = article_finder
        self.answer_query = dspy.Predict(AnswerQuerySig)
        
    def forward(self, user_query: str):
        articles = self.find_articles(user_query=user_query)
        if not articles.urls:
            return "No results found"
        
        context = [get_article_text(url=url, cursor=c) for url in articles.urls]
        answer = self.answer_query(user_query=user_query, context=context).answer
        return dspy.Prediction(answer=answer, urls=articles.urls, main_topics=articles.main_topics)

In [13]:
find_articles = RadiopaediaArticleFinder()

## Initial evaluation

In [14]:
def match_urls(ex, pred, trace=None): return set(ex.urls) == set(pred.urls)

def match_search_terms(ex, pred, trace=None): return set([x.lower() for x in ex.main_topics]) == set([x.lower() for x in pred.main_topics])

def url_recall(ex, pred, trace=None): return bool(set(ex.urls) & set(pred.urls))

In [14]:
evaluator = dspy.Evaluate(
    devset=val_examples,
    metric=url_recall, 
    display_progress=True,
    display_table=True
)

In [32]:
import tqdm
import time

scores = []

for ex in tqdm.tqdm(examples):
    pred = opt(**ex.inputs())
    scores.append(url_recall(ex, pred))
    time.sleep(1)

print(sum(scores) / len(scores))

  3%|████▌                                                                                                                                              | 1/32 [00:01<00:32,  1.05s/it]2025-07-31 07:38:11,031 - INFO - Cache hit for search query: 'ischemic penumbra'
2025-07-31 07:38:11,072 - INFO - Cache hit for search query: 'ischemic stroke'
  6%|█████████▏                                                                                                                                         | 2/32 [00:02<00:31,  1.06s/it]2025-07-31 07:38:12,108 - INFO - Cache hit for search query: 'hepatic adenoma'
2025-07-31 07:38:12,149 - INFO - Cache hit for search query: 'focal nodular hyperplasia'
 41%|███████████████████████████████████████████████████████████▎                                                                                      | 13/32 [00:14<00:22,  1.19s/it]2025-07-31 07:38:24,736 - INFO - Cache hit for search query: 'meningioma'
2025-07-31 07:38:24,775 - INFO - Cache hit for se

0.84375


In [33]:
import tqdm
import time

scores = []

for ex in tqdm.tqdm(examples): 
    pred = opt(**ex.inputs())
    scores.append(match_search_terms(ex, pred))
    #time.sleep(1)

print(sum(scores) / len(scores))

  0%|                                                                                                                                                           | 0/32 [00:00<?, ?it/s]2025-07-31 07:39:23,955 - INFO - Cache hit for search query: 'hepatic adenoma'
2025-07-31 07:39:23,976 - INFO - Cache hit for search query: 'ischemic penumbra'
2025-07-31 07:39:24,002 - INFO - Cache hit for search query: 'ischemic stroke'
2025-07-31 07:39:24,020 - INFO - Cache hit for search query: 'hepatic adenoma'
2025-07-31 07:39:24,033 - INFO - Cache hit for search query: 'focal nodular hyperplasia'
2025-07-31 07:39:24,047 - INFO - Cache hit for search query: 'Japanese encephalitis'
 12%|██████████████████▍                                                                                                                                | 4/32 [00:00<00:00, 35.44it/s]2025-07-31 07:39:24,061 - INFO - Cache hit for search query: 'CLIPPERS'
2025-07-31 07:39:24,073 - INFO - Cache hit for search query: 'callosal

0.625


In [21]:
examples[0]

Example({'user_query': 'how does a hepatic adenoma look like on MRI?', 'urls': ['https://radiopaedia.org/articles/hepatic-adenoma?lang=us'], 'main_topics': ['hepatic adenoma']}) (input_keys={'user_query'})

In [19]:
find_articles(**examples[0].inputs())

2025-07-31 07:25:57,884 - INFO - Cache hit for search query: 'hepatic adenoma'


Prediction(
    urls=['https://radiopaedia.org/articles/hepatic-adenoma?lang=us', 'https://radiopaedia.org/articles/inflammatory-hepatic-adenoma?lang=us'],
    main_topics=['hepatic adenoma']
)

In [24]:
len(examples)

32

In [31]:
from dspy.teleprompt import SIMBA

simba = SIMBA(metric=url_recall, num_threads=8, bsize=8)

opt = simba.compile(find_articles, trainset=examples[:12])

2025/07/31 07:36:07 INFO dspy.teleprompt.simba: Starting batch 1 of 8.
2025/07/31 07:36:07 INFO dspy.teleprompt.simba: Sampling program trajectories on 8 examples x 6 samples.
2025-07-31 07:36:07,909 - INFO - Cache hit for search query: 'ischemic stroke penumbra'
2025-07-31 07:36:07,911 - INFO - Cache hit for search query: 'penumbra'
2025-07-31 07:36:07,917 - INFO - Cache hit for search query: 'pneumonitis'
2025-07-31 07:36:07,923 - INFO - Cache hit for search query: 'callosal angle'
2025-07-31 07:36:07,924 - INFO - Cache hit for search query: 'ganglioglioma'


  0%|                                                                                                                                                           | 0/48 [00:00<?, ?it/s]

2025-07-31 07:36:07,925 - INFO - Cache hit for search query: 'hepatic adenoma'
2025-07-31 07:36:08,055 - INFO - Cache hit for search query: 'Japanese encephalitis'


Processed 2 / 48 examples:   2%|██▍                                                                                                                    | 1/48 [00:00<00:00, 171.47it/s]

2025-07-31 07:36:08,055 - INFO - Cache hit for search query: 'liver abscess'
2025-07-31 07:36:08,057 - INFO - Cache hit for search query: 'penumbra'


...

2025-07-31 07:37:57,860 - INFO - Cache hit for search query: 'callosal angle'


Processed 83 / 84 examples:  98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏  | 82/84 [00:02<00:00, 19.63it/s]

2025-07-31 07:37:57,874 - INFO - Cache hit for search query: 'MTA score'


Processed 84 / 84 examples: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [00:02<00:00, 32.41it/s]

2025/07/31 07:37:57 INFO dspy.teleprompt.simba: Final trainset scores: [0.75, 1.0, 1.0, 0.9166666666666666, 1.0, 0.9166666666666666, 0.9166666666666666], Best: 1.0 (at index 1)





In [34]:
opt.dump_state()

{'generate_search_query.predict': {'traces': [],
  'train': [],
  'demos': [Example({'augmented': True, 'user_query': 'how to differentiate between a hepatic adenoma and focal nodular hyperplasia on MRI?', 'reasoning': 'The user is asking how to distinguish two specific liver lesions—hepatic adenoma and focal nodular hyperplasia—based on their MRI appearances. The core topics are the two pathological entities themselves, not the imaging modality.', 'main_topics': ['hepatic adenoma', 'focal nodular hyperplasia']}) (input_keys=None)],
  'signature': {'instructions': 'Extract one or two main topics (diseases, procedures, phenomenon etc.) from the user query. \n\nDo not include imaging modalities (CT, MRI etc.) in your topics.',
   'fields': [{'prefix': 'User Query:', 'description': '${user_query}'},
    {'prefix': "Reasoning: Let's think step by step in order to",
     'description': '${reasoning}'},
    {'prefix': 'Main Topics:', 'description': '${main_topics}'}]},
  'lm': None}}

In [92]:
results = []

for ex in tqdm.tqdm(examples): 
    pred = find_articles(**ex.inputs())
    results.append({
        "query": ex.user_query,
        "urls": ex.urls,
        "pred_urls": pred.urls,
        "pred_included": url_recall(ex, pred)
    })
    time.sleep(1)

  3%|████▌                                                                                                                                              | 1/32 [00:01<00:32,  1.06s/it]2025-07-13 21:30:42,488 - INFO - Cache hit for search query: 'ischemic stroke'
2025-07-13 21:30:42,533 - INFO - Cache hit for search query: 'penumbra'
  6%|█████████▏                                                                                                                                         | 2/32 [00:02<00:32,  1.07s/it]2025-07-13 21:30:43,564 - INFO - Cache hit for search query: 'hepatic adenoma'
2025-07-13 21:30:43,598 - INFO - Cache hit for search query: 'focal nodular hyperplasia'
 16%|██████████████████████▉                                                                                                                            | 5/32 [00:05<00:28,  1.06s/it]2025-07-13 21:30:46,728 - INFO - Cache hit for search query: 'callosal angle'
2025-07-13 21:30:46,755 - INFO - Cache hit for search 

In [94]:
pd.DataFrame(results).to_excel("full_retrieval_results.xlsx")

In [19]:
qa = RadiopaediaQA(find_articles)

In [20]:
sum(h["cost"] for h in qa.history)

0

In [28]:
%%time
qa("What are the different types of endoleaks of an EVAR?")

2025-07-18 20:26:24,468 - INFO - Cache hit for url: 'https://radiopaedia.org/articles/endoleak?lang=us'


CPU times: user 418 ms, sys: 88.8 ms, total: 506 ms
Wall time: 5.9 s


Prediction(
    answer='Endoleaks after EVAR are classified into **five types**:\n\n| Type | Mechanism | Subtypes | Notes |\n|---|---|---|---|\n| **I** | **Inadequate seal at graft attachment site** | Ia – proximal, Ib – distal, Ic – iliac occluder | High-pressure leaks; require prompt treatment. |\n| **II** | **Retrograde flow via branch vessels** | IIa – single vessel, IIb – ≥2 vessels | Most common (~80 %); often self-limiting. |\n| **III** | **Mechanical failure of the graft** | IIIa – junctional separation, IIIb – graft fracture/hole | Requires urgent repair. |\n| **IV** | **Graft fabric porosity** | — | Transient; resolves spontaneously. |\n| **V** | **Endotension** | — | Sac expansion without visible leak; management controversial. |',
    urls=['https://radiopaedia.org/articles/endoleak?lang=us', 'https://radiopaedia.org/articles/type-i-endoleak?lang=us', 'https://radiopaedia.org/articles/endovascular-aneurysm-repair?lang=us', 'https://radiopaedia.org/articles/endoleak?lang=us'

In [29]:
qa.inspect_history(0)





[2025-07-18T20:23:59.542177]

System message:

Your input fields are:
1. `user_query` (str):
Your output fields are:
1. `reasoning` (str): 
2. `main_topics` (list[str]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## user_query ## ]]
{user_query}

[[ ## reasoning ## ]]
{reasoning}

[[ ## main_topics ## ]]
{main_topics}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Extract one or two main topics (diseases, procedures, phenomenon etc.) from the user query. 
        
        Do not include imaging modalities (CT, MRI etc.) in your topics.


User message:

[[ ## user_query ## ]]
What are the stages of lymphangioleiomyomatosis (LAM) on chest CT?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## main_topics ## ]]` (must be fo

In [30]:
from IPython.display import Markdown

Markdown(qa("What are the different types of endoleaks of an EVAR?").answer)

2025-07-18 20:27:20,703 - INFO - Cache hit for search query: 'endoleaks'
2025-07-18 20:27:20,724 - INFO - Cache hit for search query: 'EVAR'
2025-07-18 20:27:20,741 - INFO - Cache hit for url: 'https://radiopaedia.org/articles/endoleak?lang=us'
2025-07-18 20:27:20,741 - INFO - Cache hit for url: 'https://radiopaedia.org/articles/type-i-endoleak?lang=us'
2025-07-18 20:27:20,742 - INFO - Cache hit for url: 'https://radiopaedia.org/articles/endovascular-aneurysm-repair?lang=us'
2025-07-18 20:27:20,742 - INFO - Cache hit for url: 'https://radiopaedia.org/articles/endoleak?lang=us'


Endoleaks after EVAR are classified into **five types**:

| Type | Mechanism | Subtypes | Notes |
|---|---|---|---|
| **I** | **Inadequate seal at graft attachment site** | Ia – proximal, Ib – distal, Ic – iliac occluder | High-pressure leaks; require prompt treatment. |
| **II** | **Retrograde flow via branch vessels** | IIa – single vessel, IIb – ≥2 vessels | Most common (~80 %); often self-limiting. |
| **III** | **Mechanical failure of the graft** | IIIa – junctional separation, IIIb – graft fracture/hole | Requires urgent repair. |
| **IV** | **Graft fabric porosity** | — | Transient; resolves spontaneously. |
| **V** | **Endotension** | — | Sac expansion without visible leak; management controversial. |

## Optimization

In [170]:
from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric=match_urls, teacher_settings=dict(lm=gpt41), max_rounds=3)

opt = teleprompter.compile(find_articles, trainset=train_examples)

  0%|                                                                                                                                                            | 0/6 [00:00<?, ?it/s]2025-07-06 20:16:32,809 - INFO - Cache hit for search query: 'liver abscess'
2025-07-06 20:16:36,590 - INFO - Cache hit for search query: 'liver abscess'
 17%|████████████████████████▋                                                                                                                           | 1/6 [00:10<00:52, 10.50s/it]2025-07-06 20:16:40,463 - INFO - Cache hit for search query: 'ganglioglioma'
2025-07-06 20:16:42,344 - INFO - Cache hit for search query: 'ganglioglioma'
2025-07-06 20:16:46,424 - INFO - Cache hit for search query: 'ganglioglioma'
 33%|█████████████████████████████████████████████████▎                                                                                                  | 2/6 [00:20<00:40, 10.15s/it]2025-07-06 20:17:42,268 - INFO - Cache hit for search query: 'sept

Bootstrapped 0 full traces after 5 examples for up to 3 rounds, amounting to 18 attempts.


In [34]:
evaluator = dspy.Evaluate(
    devset=examples,
    metric=match, 
    num_threads=5,
    display_progress=True,
    display_table=True
)

In [37]:
evaluator(opt)

  0%|                                                                                                                                                           | 0/32 [00:00<?, ?it/s]

2025-07-05 13:55:54,721 - INFO - Cache hit for search query: 'hepatic adenoma'
2025/07/05 13:55:54 ERROR dspy.utils.parallelizer: Error for Example({'user_query': 'how does japanese encephalitis look like on MRI?', 'urls': ['https://radiopaedia.org/articles/japanese-encephalitis?lang=us'], 'main_topics': ['japanese encephalitis']}) (input_keys={'user_query'}): Re-using a cursor inside a query by that query is not allowed. Set `provide_traceback=True` for traceback.
2025-07-05 13:55:54,725 - INFO - Cache hit for search query: 'CLIPPERS'


Average Metric: 0.00 / 0 (0%):   3%|███▋                                                                                                                | 1/32 [00:01<00:52,  1.70s/it]

2025-07-05 13:55:55,151 - INFO - Cache hit for search query: 'hepatic adenoma'
2025-07-05 13:55:55,403 - INFO - Cache hit for search query: 'ischemic stroke'
2025-07-05 13:55:56,239 - INFO - Retrying request to /chat/completions in 0.448564 seconds


Average Metric: 2.00 / 2 (100.0%):   9%|██████████▌                                                                                                     | 3/32 [00:04<00:34,  1.20s/it]

...

2025-07-05 13:56:26,709 - INFO - Cache hit for search query: 'multiple sclerosis'


Average Metric: 17.00 / 31 (54.8%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:35<00:00,  1.11s/it]

2025/07/05 13:56:28 INFO dspy.evaluate.evaluate: Average Metric: 17.0 / 32 (53.1%)


,user_query,example_urls,example_main_topics,pred_urls,pred_main_topics,match,urls,main_topics
0,how does a hepatic adenoma look like on MRI?,[https://radiopaedia.org/articles/hepatic-adenoma?lang=us],[hepatic adenoma],[https://radiopaedia.org/articles/hepatic-adenoma?lang=us],[hepatic adenoma],✔️ [True],NaN,NaN
1,penumbra ischemic stroke perfuson imaging,[https://radiopaedia.org/articles/ct-perfusion-in-ischaemic-stroke...,[penumbra perfusion imaging],[https://radiopaedia.org/articles/ischaemic-stroke-summary?lang=us...,"[ischemic stroke, penumbra, perfusion imaging]",,NaN,NaN
2,how to differentiate between a hepatic adenoma and focal nodular h...,"[https://radiopaedia.org/articles/hepatic-adenoma?lang=us, https:...","[hepatic adenoma, focular nodular hyperplasia]","[https://radiopaedia.org/articles/hepatic-adenoma?lang=us, https:/...","[hepatic adenoma, focal nodular hyperplasia]",,NaN,NaN
3,how does japanese encephalitis look like on MRI?,NaN,NaN,NaN,NaN,,[https://radiopaedia.org/articles/japanese-encephalitis?lang=us],[japanese encephalitis]
4,what is CLIPPERS?,[https://radiopaedia.org/articles/chronic-lymphocytic-inflammation...,[CLIPPERS],[https://radiopaedia.org/articles/chronic-lymphocytic-inflammation...,[CLIPPERS],✔️ [True],NaN,NaN
5,"What is a normal callosal angle, and how do I measure it?",[https://radiopaedia.org/articles/callosal-angle?lang=us],[callosal angle],[https://radiopaedia.org/articles/callosal-angle?lang=us],[callosal angle],✔️ [True],NaN,NaN
6,Can you tell me more about the MTA score?,[https://radiopaedia.org/articles/medial-temporal-lobe-atrophy-sco...,[MTA score],[https://radiopaedia.org/articles/medial-temporal-lobe-atrophy-sco...,[MTA score],✔️ [True],NaN,NaN
7,what are the criteria of an liver abscess,[https://radiopaedia.org/articles/hepatic-abscess-1?lang=us],[liver abscess],[https://radiopaedia.org/articles/hepatic-abscess-1?lang=us],[liver abscess],✔️ [True],NaN,NaN
8,pneumonitis,[https://radiopaedia.org/articles/pneumonitis?lang=us],[pneumonitis],[https://radiopaedia.org/articles/pneumonitis?lang=us],[pneumonitis],✔️ [True],NaN,NaN
9,how does the penumbra look in ct perfusion?,[https://radiopaedia.org/articles/ischaemic-penumbra?lang=us],[penumbra ct perfusion],[https://radiopaedia.org/articles/ischaemic-penumbra?lang=us],[penumbra],✔️ [True],NaN,NaN


53.12

In [116]:
opt(user_query="How to distinguish type 1 and type 2 endoleaks?")

no matching search result found for user query: 'How to distinguish type 1 and type 2 endoleaks?' and search term: 'Type 1 and type 2 endoleaks'
no matching search result found for user query: 'How to distinguish type 1 and type 2 endoleaks?' and search term: 'Type 1 endoleak Type 2 endoleak'


Prediction(
    notes=["no matching search result found for user query: 'How to distinguish type 1 and type 2 endoleaks?' and search term: 'Type 1 and type 2 endoleaks'", "no matching search result found for user query: 'How to distinguish type 1 and type 2 endoleaks?' and search term: 'Type 1 endoleak Type 2 endoleak'"],
    urls=[],
    hops=1,
    main_topics=['Type 1 and type 2 endoleaks', 'Type 1 endoleak Type 2 endoleak']
)

In [287]:
qa = RadiopaediaQA(cursor=c, article_finder=optimized_finder)

In [288]:
qa("cerebral vein thrombosis mri")

2025-06-26 20:02:08,042 - INFO - Cache hit for search query: 'cerebral vein thrombosis'
2025-06-26 20:02:11,491 - INFO - Cache hit for url: 'https://radiopaedia.org/articles/cerebral-venous-thrombosis?lang=us'


Prediction(
    answer='Cerebral vein thrombosis (CVT) can be evaluated using MRI, which is effective in visualizing both the thrombus and its sequelae. On MRI:\n\n- In the acute phase, the clot appears isointense on T1-weighted images and hypointense on T2-weighted images, which can mimic a flow void.\n- In the subacute phase, the clot becomes hyperintense on T1-weighted images.\n- FLAIR sequences may show sulcal hyperintensity indicating localized subarachnoid hemorrhage.\n- Post-contrast T1-weighted images (T1 C+) may demonstrate focal pachymeningeal and leptomeningeal enhancement due to elevated venous pressure upstream of the thrombosis.\n- Diffusion-weighted imaging (DWI) and apparent diffusion coefficient (ADC) sequences may show diffusion restriction in chronic stages of the clot.\n- MR venography (MRV), especially contrast-enhanced MRV, is used to demonstrate absence of flow in the affected venous sinuses. 2D time-of-flight (TOF) MRV is commonly performed but less sensitive th

In [289]:
with open("../test_queries.txt", "r") as f:
    queries = f.readlines()

In [290]:
for query in queries[-10:]:
    print(query)
    print(qa(query))
    print("-"*100)

what are the CT findings of fat necrosis in the breast?



2025-06-26 20:02:40,174 - INFO - Cache hit for url: 'https://radiopaedia.org/articles/fat-necrosis-breast-2?lang=us'


Prediction(
    answer='The CT findings of fat necrosis in the breast are not explicitly detailed in the provided context. However, based on the described pathology and imaging features on mammography and ultrasound, fat necrosis typically appears as an area of fat disruption with possible calcifications. On mammography, it can present as an ill-defined, irregular, spiculated mass-like area with peripheral stippled curvilinear calcifications, sometimes evolving into a well-circumscribed oil cyst with fine curvilinear wall calcifications. Ultrasound findings vary with the phase: acute phase shows increased echogenicity due to edema; subacute phase shows an ill-defined complex cystic lesion with low-level echoes; late phase shows calcified walls with posterior shadowing. While CT is not specifically described, fat necrosis would likely appear as a fat-density lesion with possible calcifications and surrounding fibrosis or inflammation.',
    urls=['https://radiopaedia.org/articles/fat-ne

2025-06-26 20:03:25,783 - INFO - Cache hit for search query: 'tension pneumothorax'


Prediction(
    answer='On a chest X-ray, a tension pneumothorax can be recognized by several radiographic features indicating increased intrapleural pressure on the affected side. These include:\n\n- Ipsilateral hyper-expansion of the hemithorax with splayed ribs\n- Increased translucency (hypertransradiancy) on the affected side\n- Collapse of the ipsilateral lung\n- Depression of the ipsilateral diaphragm\n- Contralateral mediastinal shift away from the affected side\n\nNote that the diagnosis of tension pneumothorax is primarily clinical and should not be delayed for radiographic confirmation, as it is a medical emergency requiring urgent decompression.',
    urls=['https://radiopaedia.org/articles/tension-pneumothorax?lang=us'],
    main_topics=['tension pneumothorax']
)
----------------------------------------------------------------------------------------------------
How does a TOF MRA work?

Prediction(
    answer='Time-of-flight (TOF) MR angiography works by exploiting the in

2025-06-26 20:03:42,205 - INFO - Cache hit for search query: 'neuroborreliosis'
2025-06-26 20:03:44,472 - INFO - Cache hit for url: 'https://radiopaedia.org/articles/lyme-disease?lang=us'


Prediction(
    answer='On MRI, neuroborreliosis (neurological Lyme disease) can show several characteristic findings depending on the specific neurological involvement:\n\n- Cranial neuritis: Enhancement of cranial nerve roots on T1 post-contrast (Gd), most commonly affecting the facial nerve (cranial nerve VII), sometimes bilaterally, and can also involve cranial nerves VIII, V, IV, and III.\n- Meningitis: Leptomeningeal enhancement on T1 post-contrast, which can rarely extend over the spinal cord.\n- Encephalitis: Small, nonspecific foci of T2 hyperintensity in periventricular, subcortical, or brainstem regions, typically without enhancement. Less commonly, larger tumefactive T2 hyperintense lesions or CLIPPERS-like enhancement patterns may be seen.\n- Myelitis: Longitudinally extensive or short segment T2 hyperintense lesions in the spinal cord, often centrally or anteriorly located, which may or may not enhance post-contrast.\n- Radiculoneuritis: Increased signal on fluid-sensitiv

2025-06-26 20:03:54,714 - INFO - Cache hit for search query: 'Baker cyst'
2025-06-26 20:03:57,344 - INFO - Cache hit for url: 'https://radiopaedia.org/articles/baker-cyst-2?lang=us'


Prediction(
    answer='A Baker cyst, also known as a popliteal cyst, is a fluid-filled swelling that occurs in the popliteal fossa (the area behind the knee) between the medial head of the gastrocnemius muscle and the semimembranosus tendon. It forms due to a communication between the knee joint and the gastrocnemius-semimembranosus bursa, causing synovial fluid to accumulate and distend this area. Although called a cyst, it is not a true cyst or bursa but rather a distended synovial-lined lesion. Baker cysts are often found incidentally during knee imaging and can be associated with conditions like degenerative arthritis or rheumatoid arthritis. They may cause symptoms if they rupture or compress nearby structures, leading to pain or swelling behind the knee.',
    urls=['https://radiopaedia.org/articles/baker-cyst-2?lang=us'],
    main_topics=['Baker cyst']
)
----------------------------------------------------------------------------------------------------
how does a clivus chordo

In [291]:
qa.inspect_history(0)





[2025-06-26T20:02:08.038552]

System message:

Your input fields are:
1. `user_query` (str):
Your output fields are:
1. `reasoning` (str): 
2. `main_topics` (list[str]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## user_query ## ]]
{user_query}

[[ ## reasoning ## ]]
{reasoning}

[[ ## main_topics ## ]]
{main_topics}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Extract one or two main topics (diseases, procedures, phenomenon etc.) from the user query


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## user_query ## ]]
ms lesions on mri typical findings


Assistant message:

[[ ## reasoning ## ]]
Not supplied for this particular example. 

[[ ## main_topics ## ]]
["multiple sclerosis"]


User message:

This is an examp